In [1]:
import os
import json
import boto3
import pandas as pd
import numpy as np
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer
from boto3.dynamodb.conditions import Key
from decimal import Decimal
from dotenv import load_dotenv
from datetime import datetime
import json

load_dotenv()

True

In [2]:

ts= TypeSerializer()
td = TypeDeserializer()

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

In [3]:
def query_sql(beer_id, dynamodb=None):
    if not dynamodb:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))
    stmt = "SELECT BREW_ID,PI_TMSTP,STAGE,location,temperature,thermo_loc,thermo_temp FROM BREW_MON WHERE BREW_ID='" + beer_id + "'  ORDER BY PI_TMSTP DESC"
   # print(lambda_handler(event=brew1))
    response = dyna_clnt.execute_statement(Statement=stmt)
    #print("HI")
    return response    
 

In [4]:
lines = query_sql(beer_id='Amber Ale')
df_raw = pd.json_normalize(lines, record_path =['Items'])

In [5]:
    df_raw['pi_time']=pd.to_datetime(df_raw['PI_TMSTP.S'])
    df_raw['temperature_n']=pd.to_numeric(df_raw['temperature.S'], downcast="float")
    df_raw['thermo_temp_n']=pd.to_numeric(df_raw['thermo_temp.N'], downcast="float")    
    df_raw.set_index('pi_time',inplace=True)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n > 50].index)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n < 0].index)
    df_raw = df_raw.drop(columns=['BREW_ID.S','PI_TMSTP.S','STAGE.S','location.S','thermo_loc.S','temperature.S','thermo_temp.N'])


In [6]:
    df_4h = df_raw.groupby([pd.Grouper(freq='2H', level='pi_time')]).mean(numeric_only=True)
    df_4h = df_4h[df_4h['temperature_n'].notna()]
    df_4h = df_4h[df_4h['thermo_temp_n'].notna()]
    df_4h['temperature_n'] = df_4h['temperature_n'].astype('float64').round(decimals=4)
    df_4h['thermo_temp_n'] = df_4h['thermo_temp_n'].astype('float64').round(decimals=4)
  
 

In [7]:
#print(df_4h)

In [8]:
df_4h=df_4h.reset_index()
print(df_4h.info())
df_4h['pi_time'] = df_4h['pi_time'].astype(str)
#df_4h = df_4h.drop(columns=['pi_time'])
print('----------------------NEW INFO-----------------------------------------')
print(df_4h.info())
print('----------------------DATA-----------------------------------------')
print(df_4h)
#df_4h['pi_s'] = df_4h['pi_time'].strftime('%Y-%m-%d %X')
#df_4h['pi_time'] = df_4h['pi_time'].strftime('%Y-%m-%d %X')
#df_4h = df_4h.drop(columns=['pi_time'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pi_time        39 non-null     datetime64[ns]
 1   temperature_n  39 non-null     float64       
 2   thermo_temp_n  39 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.0 KB
None
----------------------NEW INFO-----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pi_time        39 non-null     object 
 1   temperature_n  39 non-null     float64
 2   thermo_temp_n  39 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.0+ KB
None
----------------------DATA-----------------------------------------
                pi_time  temperature_n  thermo_temp_n
0   20

In [9]:
# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))

table = dynamodb.Table('BREW_LOG')

In [10]:
#convert to JSON dictionary
items_from_raw = df_4h.to_dict(orient='records')

#reload to convert to decimal
changed_data_raw = json.loads(json.dumps(items_from_raw), parse_float=Decimal)

#LOAD to AWS
table.update_item(
   Key={"BEER_ID":"Amber Ale", "BEER_DATE": "2020-07-19"},
                UpdateExpression="set TEMP_DATA_2H=:r",
                ExpressionAttributeValues={":r": changed_data_raw},
                ReturnValues="UPDATED_NEW"
)


{'Attributes': {'TEMP_DATA_2H': [{'pi_time': '2020-07-20 02:00:00',
    'thermo_temp_n': Decimal('27.58'),
    'temperature_n': Decimal('21')},
   {'pi_time': '2020-07-20 06:00:00',
    'thermo_temp_n': Decimal('27.49'),
    'temperature_n': Decimal('21')},
   {'pi_time': '2020-07-20 18:00:00',
    'thermo_temp_n': Decimal('28.13'),
    'temperature_n': Decimal('21')},
   {'pi_time': '2020-07-21 00:00:00',
    'thermo_temp_n': Decimal('28.95'),
    'temperature_n': Decimal('20.5')},
   {'pi_time': '2020-07-21 08:00:00',
    'thermo_temp_n': Decimal('25.96'),
    'temperature_n': Decimal('19')},
   {'pi_time': '2020-07-22 10:00:00',
    'thermo_temp_n': Decimal('19.14'),
    'temperature_n': Decimal('19.0435')},
   {'pi_time': '2020-07-22 16:00:00',
    'thermo_temp_n': Decimal('19.67'),
    'temperature_n': Decimal('20')},
   {'pi_time': '2020-07-23 02:00:00',
    'thermo_temp_n': Decimal('19.32'),
    'temperature_n': Decimal('19.3333')},
   {'pi_time': '2020-07-23 04:00:00',
    'the

In [ ]:
#Test Data
data2 = {
    'temperature': [60.3, 62.2345, 65.2345, 63.4567, 61.4567],
    'timestamp': [
        '2023-12-18 00:00:00',
        '2023-12-18 02:00:00',
        '2023-12-18 04:00:00',
        '2023-12-18 06:00:00',
        '2023-12-18 08:00:00'
    ]
}
#Convert to Data Frame
df_tss = pd.DataFrame(data2)

#convert to JSON dictionary
items_from_test = df_tss.to_dict(orient='records')

#reload to convert to decimal
changed_data = json.loads(json.dumps(items_from_test), parse_float=Decimal)

#LOAD to AWS
table.update_item(
   Key={"BEER_ID":"BAD_SANTA", "BEER_DATE": "2021-11-20"},
                UpdateExpression="set TEMP_DATA_TEST_2023_12_21_v=:r",
                ExpressionAttributeValues={":r": changed_data},
                ReturnValues="UPDATED_NEW"
)

#df = pd.DataFrame(changed_data)



# Convert DataFrame to a list of dictionaries
#items_from_test = df.to_dict(orient='records',parse_float=Decimal)
#items_from_test = pd.io.json.dumps(df, double_precision=2)

In [ ]:
print(df_tss)
#print(changed_data)
#print(df)
print(df_4h)